<a href="https://colab.research.google.com/github/WetSuiteLeiden/example-notebooks/blob/main/datasets/gemeentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# (only) in colab, run this first to install wetsuite from (the most recent) source. 
#    (this should soon simplify to something like   !pip3 install --upgrade wetsuite)
# For your own setup, see wetsuite's install guidelines.
!pip3 install -U wetsuite

# Purpose of this notebook

A quick introduction of the more-of-a-table-than-a-dataset that is some metadata of the Dutch municipalities

In [11]:
import pprint, random, datetime, re

import pandas

import wetsuite.datasets

In [3]:
cons = wetsuite.datasets.load('internetconsultaties-partial-struc')

print( cons.description )


The text contents of a good portion of the answers at https://www.internetconsultatie.nl
The focus is the reactions themselves, but varied

More time should be spent on anonymizing this, 
and on dealing with the PDFs (where the anonymization is likely more relevant).

For example:                              
  {
    'entry_page_url': 'https://www.internetconsultatie.nl/registratieregeling/b1',
    'details': {'title': 'Wijziging Regeling periodieke registratie Wet BIG (orthopedagoog-generalist en klinisch technoloog)',
      'labels': ['Consultatie gesloten', 'Zorg en gezondheid'],
      'overview': {'Startdatum consultatie': '16-05-2024',
        'Einddatum consultatie': '13-06-2024',
        'Status': 'Gesloten',
        'Type consultatie': 'Ministeriële regeling',
        'Organisatie': 'Ministerie van Volksgezondheid, Welzijn en Sport',
        'Onderwerpen': 'Organisatie en beleid'},
      'paragraphs': {'In het kort': 'In 2020 zijn twee nieuwe beroepen opgenomen in artikel 3 v

In [8]:
summary = []

cons_items = list( cons.data.items() )
random.shuffle( cons_items )

for case_id, dd in cons_items:
    print( dd.keys() )
    pprint.pprint(dd)
    break

dict_keys(['entry_page_url', 'details', 'reacties'])
{'details': {'documenten': [['Verslag over de resultaten van de consultatie',
                             'https://www.internetconsultatie.nl/kapitaalvereisten2020/document/5775'],
                            ['Concept regeling',
                             'https://www.internetconsultatie.nl/kapitaalvereisten2020/document/5436'],
                            ['IAK',
                             'https://www.internetconsultatie.nl/kapitaalvereisten2020/document/5438']],
             'labels': [],
             'overview': {'Einddatum consultatie': '11-03-2020',
                          'Keten-ID': '10874',
                          'Onderwerpen': 'Financieel toezicht',
                          'Organisatie': 'Ministerie van Financiën',
                          'Publicatiedatum': '12-02-2020',
                          'Status': 'Resultaat gepubliceerd',
                          'Type consultatie': 'Wet'},
             'paragraphs

In [51]:
def overview_mindate( oveview ):
    dates = []
    for key, value in overview.items():
        if 'datum' in key.lower() and re.match('[0-9][0-9]-[0-9][0-9]-[0-9][0-9][0-9][0-9]', value) is not None:
            day, month, year = value.split('-')
            dates.append( datetime.date( int(year), int(month), int(day) ) )
    if len(dates) == 0:
        pprint.pprint(d)
    return min(dates)


# pandas is a nice way to inspect (and manipulate) things in a tabular-like way
# whereas the data itself is more structured (and e.g. 'overview' is more free-form)
# pandas prefers relatively flat dict or list, so...

summary = []

for case_id, casedict in cons_items:
    # casedict has keys:                 'entry_page_url', 'details', 'reacties
    # details is a dict which has keys:  'title', 'overview', 'labels', 'documenten', 'paragraphs' 

    flat = {}
    flat['id']             = case_id
    flat['entry_page_url'] = casedict['entry_page_url']

    #if 'details' not in casedict:
    #    print('SKIP WEIRD - NO DETAILS')
    #    continue
    details = casedict['details'] # has kets

    #if 'overview' not in details:
    #    print('SKIP WEIRD - NO OVERVIEW IN DETAILS')
    #    continue
    overview = details['overview'] # has kets

    flat['labels']  = details['labels']
    flat['mindate'] = overview_mindate( overview )
    flat.update(overview)

    flat['num_responses'] = len( casedict['reacties'] )

    #flat.update( details )
    #flat.update( details.pop('overview') )
    #flat.update( details )

    summary.append( flat )
    #print(d)
    #break


# sorting a pandas dataframe may be easier, actually
summary.sort( key = lambda x:x['num_responses'], reverse=True)
#summary.sort( key = lambda x:x['mindate'], reverse=True)
df = pandas.DataFrame( summary )
pandas.set_option('display.max_rows', 200)
df

,id,entry_page_url,labels,mindate,Startdatum consultatie,Einddatum consultatie,Status,Type consultatie,Organisatie,Onderwerpen,num_responses,Publicatiedatum,Keten-ID
0,vuurwerkverbod,https://www.internetconsultatie.nl/vuurwerkver...,"[Consultatie gesloten, Natuur en milieu, Openb...",2022-01-04,04-01-2022,04-02-2022,Gesloten,Initiatiefwet,Tweede Kamer der Staten-Generaal,Geluid Lucht Politie brandweer en hulpdiens...,11964,NaN,NaN
1,inschrijftermijnbrpvreemdelingen,https://www.internetconsultatie.nl/inschrijfte...,[],2016-12-03,NaN,03-02-2017,Gesloten,AMvB,Ministerie van Binnenlandse Zaken en Koninkrij...,Gemeenten Rijksoverheid De Nederlandse Antille...,7564,03-12-2016,NaN
2,wetnatuur,https://www.internetconsultatie.nl/wetnatuur,[],2011-10-06,NaN,18-11-2011,Resultaat gepubliceerd,Wet,Ministerie van Economische Zaken en Klimaat,Natuur- en landschapsbeheer,4714,06-10-2011,NaN
3,intrekkingzondagswet,https://www.internetconsultatie.nl/intrekkingz...,[],2016-04-20,NaN,01-06-2016,Resultaat gepubliceerd,Wet,Ministerie van Binnenlandse Zaken en Koninkrij...,Gemeenten Provincies Rijksoverheid,4140,20-04-2016,NaN
4,wetsvoorsteltestbewijzen,https://www.internetconsultatie.nl/wetsvoorste...,[],2021-03-08,NaN,15-03-2021,Gesloten,Wet,Ministerie van Binnenlandse Zaken en Koninkrij...,Gezondheidsrisico's,3543,08-03-2021,12653
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,bestuursrechtspraak,https://www.internetconsultatie.nl/bestuursrec...,[],2019-10-10,NaN,05-12-2019,Gesloten,Wet,Ministerie van Justitie en Veiligheid,Bestuursrecht,0,10-10-2019,10667
2752,middenhuur,https://www.internetconsultatie.nl/middenhuur,[],2018-03-13,NaN,09-04-2018,Gesloten,Wet,Ministerie van Binnenlandse Zaken en Koninkrij...,Bouwen en verbouwen Huren en verhuren,0,13-03-2018,NaN
2753,bromilieuhygienischonderzoek,https://www.internetconsultatie.nl/bromilieuhy...,"[Consultatie gesloten, Bestuur]",2023-09-05,05-09-2023,30-10-2023,Gesloten,AMvB,Ministerie van Binnenlandse Zaken en Koninkrij...,Organisatie en beleid,0,NaN,14423
2754,owms,https://www.internetconsultatie.nl/owms,[],2022-06-24,NaN,24-07-2022,Gesloten,Beleidsnota,Ministerie van Binnenlandse Zaken en Koninkrij...,Organisatie en beleid ICT Organisatie en beleid,0,24-06-2022,NaN


In [ ]:
# attempting to cherry pick an interesting case with a moderate amount of reactions...
for case_id, casedict in cons.data.items():
    if len(casedict['reacties']) > 5  and  len(casedict['reacties']) < 50:
        print(case_id)

In [48]:
# a few reactions
cherry_picked = cons.data['eigen_risico']
for from_url, type, reactie_text in cherry_picked['reacties']:
    print( reactie_text )
    print( '---' )

Ik wil graag de aanname in twijfel trekken dat mensen die gezond zijn, en dus minder of geen gebruik maken van het eigen risico, alleen maar gezond zijn door puur geluk (zoals in de tekst staat aangegeven).
Ik begrijp volledig dat iedereen ziek kan worden, getroffen door een aandoening, of iets anders ergs.
Tegelijkertijd is gezond zijn ook het resultaat van levensstijl. Zelf ben ik iemand die aardig zijn best doet om gezond te blijven. Op tijd naar bed, gezond eten, voldoende bewegen, dat soort dingen.
Er zijn ook veel mensen die hier niet mee bezig zijn en volop roken, drinken en ongezond eten, niet bewegen etc. Dit is natuurlijk hun vrije keuze en die mogen ze maken. Alleen, het frustreert mij wel dat hierdoor voor deze mensen en dus voor de samenleving als geheel de zorgkosten omhoog gaan. Daar betaal ik ook aan mee en dát vind ík niet solidair noch fair.
Het eigen risico mag van mij dus blijven bestaan. Het niet hoeven gebruiken van zorg voelt voor mij dan als een soort beloning v